<a href="https://colab.research.google.com/github/Ditsuhi/Nitrogen_Dioxide_Prediction/blob/main/BiConvLSTM_NO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all required libraries

import zipfile
from glob import glob
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from scipy.interpolate import NearestNDInterpolator
from keras.models import Sequential
from keras.layers import ConvLSTM2D, BatchNormalization
from keras.layers import Bidirectional
from keras.layers.convolutional import  Conv2D 


In [ ]:
# To calculate nearest neighbor interpolation for meteorological data

def CalcNNvalue(array_interpolate):
  
  array_float = array_interpolate.astype(float)
  knowncell_position= np.argwhere(array_float!=0)  
  knowncell_value = array_float[array_float!=0] 
  unknowncell_position = np.argwhere(array_float==0)
  myInterpolator = NearestNDInterpolator(knowncell_position, knowncell_value) 
  unknown_values = myInterpolator(unknowncell_position)
  array_float[array_float == 0 ] = unknown_values
  return array_float.tolist()


def calc_NN_fullData(full_data):
  NN_list =[]
  for item in full_data:    
    try: 
      NN_list.append(CalcNNvalue(item))
    except IndexError:
      NN_list.append(item.tolist())  
  return NN_list


def calculate_NN_fullData_allAttributes (df_all):
  df_all_NN_list = []
  # The number in the range is the number of meteorological features 
  # to be interpolated using nearest neighbor interpolation.  
  for attr_numb in range(7): 
    certain_attr = df_all[:, :, attr_numb]    
    certain_attr_reshaped= certain_attr.reshape(certain_attr.shape[0], 20, 17)
    certain_attr_reshaped_NN = calc_NN_fullData(certain_attr_reshaped) 
    certain_attr_reshaped_NN_original_shape = np.reshape(certain_attr_reshaped_NN, (certain_attr.shape[0], 340))
    df_all_NN_list.append(certain_attr_reshaped_NN_original_shape.tolist())   
  df_all_NN_array = np.dstack((item)for item in df_all_NN_list)  
  return df_all_NN_array



In [ ]:
#unzip data giving the path of certain dataset

path = '/content/AirMetTrafMadridDataCSV_2019.zip'
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

airMetTraf = glob("/content/*.csv")

In [ ]:
#sort dataset with chronological order

def sortingFiles(eachFile):
    return int(eachFile) if eachFile.isdigit() else eachFile
def natural_keys(eachFile):
    return [sortingFiles(c) for c in re.split('(\d+)',eachFile)]

sorted_airMetTraf= sorted(airMetTraf, key = natural_keys)
sorted_airMetTraf_2019 = sorted_airMetTraf[:4344]
#sorted_airMetTraf_2020 = sorted_airMetTraf[4344:]


# These are the feyures from the matrices: FID	 NO2	 UV	 windSpeed	 windDir	 Temp	 Humidity	 Pressure	 SolarRad	 Prec	 intensidad	 ocupacion	 carga	 vmed

df = [pd.read_csv(f, usecols=[' NO2', ' UV',  ' windSpeed', ' windDir', ' Temp', ' Humidity', ' Pressure', ' SolarRad', ' Prec', ' intensidad',	' ocupacion',	' carga',	 ' vmed']).values for f in sorted_airMetTraf_2019]


df_all  = np.asarray(df)

# This step is for outlier handling (Temperature:res; Humidity:reshum;
# and Average Speed:speed)
res = np.where(df_all[:, :, 4] < -3)
reshum = np.where(df_all[:, :, 5] < 0)
speed = np.where(df_all[:, :, 12] < 0)

print(len(res[1]))
print(len(reshum[1]))
print(len(speed[0]))

# all values for a temperature data below -3 are converted to an average
# before and after the values.

for i in range(len(res[0])):
  if df_all[:, :, 4][res[0][i]][res[1][i]-1] > -3 and df_all[:, :, 4][res[0][i]][res[1][i]+1] > -3:
    df_all[:, :, 4][res[0][i]][res[1][i]] = (df_all[:, :, 4][res[0][i]][res[1][i]-1]+df_all[:, :, 4][res[0][i]][res[1][i]+1])/2


# all values for a humidity data below 0 are converted to an average
# before and after the values.

for i in range(len(reshum[0])):
  if df_all[:, :, 5][reshum[0][i]][reshum[1][i]-1] >= 0 and df_all[:, :, 5][reshum[0][i]][reshum[1][i]+1] >= 0:
    df_all[:, :, 5][reshum[0][i]][reshum[1][i]] = (df_all[:, :, 5][reshum[0][i]][reshum[1][i]-1]+df_all[:, :, 5][reshum[0][i]][reshum[1][i]+1])/2


# all values for a speed data below 0 are converted to 0.

for i in range(len(speed[0])):
  df_all[:, :, 12][speed[0][i]][speed[1][i]] = 0


# deleting precipitation as the majority of the values are equal to 0.

df_all_non_prec = np.delete(df_all, 8, 2)
air= df_all_non_prec[:, :, 0].reshape(-1, 340, 1)
traf =  df_all_non_prec[:, :, 8:12].reshape(-1, 340, 4)
NN_dataframe = calculate_NN_fullData_allAttributes (df_all_non_prec[:, :, 1:8])
df_air_NN_Met = np.concatenate((air, idw_dataframe, traf), axis=2)
not_nun = np.nan_to_num(df_air_NN_Met)
round_data = np.round(not_nun, 1)


820
3
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [ ]:
round_data.shape

(4344, 340, 12)

In [ ]:
# split dataset to X and y (dependent and independent)

def split_sequence(sequence, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
   
		# find the end of this pattern
		end_ix = i + 12
    
		# check if we are beyond the sequence
		if end_ix+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern    
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix+time_steps]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 

# define input sequence
raw_seq = round_data
# choose a number of time steps (there are two case of time lags: 6-hour and 12-hour)
time_steps = 6
X, y = split_sequence(raw_seq, time_steps)


In [ ]:
y.shape

(4326, 340, 12)

In [ ]:
#split data to train and test sets

X_train_notNorm, X_test_notNorm, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)

In [ ]:
y_test.shape

(866, 340, 12)

In [ ]:
# to normalise train data using MinMaxScaler
number_selected_columns = 12

scaler = MinMaxScaler(feature_range=(0, 1), copy = False)
train_Normalised = X_train_notNorm.reshape(-1, 340*number_selected_columns)
test_Normalised = X_test_notNorm.reshape(-1, 340*number_selected_columns)

train_scaled = scaler.fit_transform(train_Normalised)
test_scaled = scaler.transform(test_Normalised)

X_train = train_scaled.reshape(X_train_notNorm.shape[0], X_train_notNorm.shape[1], X_train_notNorm.shape[2], X_train_notNorm.shape[3])
X_test = test_scaled.reshape(X_test_notNorm.shape[0], X_test_notNorm.shape[1], X_test_notNorm.shape[2], X_test_notNorm.shape[3])

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 20, 17*number_selected_columns, 1))
y_train_reshaped = y_train.reshape((y_train.shape[0], 20, 17*number_selected_columns, 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 20, 17*number_selected_columns, 1))
y_test_reshaped = y_test.reshape(y_test.shape[0], 20, 17*number_selected_columns, 1)

In [ ]:
X_train_reshaped.shape

(3460, 12, 20, 204, 1)

In [ ]:
# In the following two cells are the code for parameter optimisation.
# First of all, the class was created in order to split data.

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


btscv = BlockingTimeSeriesSplit(n_splits=3)
model = KerasRegressor(build_fn=create_model, verbose=0)


#define the grid search parameters


optimizer = ['RMSprop',  'Adam']
kernel_size = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]
filters= [8, 16]
merge_mode=['sum', 'concat', 'ave']

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
param_grid = dict(filters=filters,  kernel_size=kernel_size, optimizer=optimizer, merge_mode= merge_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=btscv)
grid_result = grid.fit(X_train_reshaped, y_train_reshaped, epochs = 20)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# define the architecture of the proposed model

def create_model(number_selected_columns=12, optimizer='adam', kernel_size=(9, 9), filters=8, merge_mode="concat"):
    
    model = Sequential()    
    model.add(Bidirectional(ConvLSTM2D(input_shape=(None, 20, 17*number_selected_columns, 1),  filters=filters,  kernel_size=kernel_size, padding="same", return_sequences=True), merge_mode=merge_mode))
    
    model.add(BatchNormalization())  
    model.add(Bidirectional(ConvLSTM2D(filters=filters, kernel_size=kernel_size, padding="same", return_sequences=True), merge_mode=merge_mode))    
    model.add(BatchNormalization())
    model.add(Bidirectional(ConvLSTM2D(filters=filters,  kernel_size=kernel_size, padding="same"), merge_mode=merge_mode))     
    model.add(BatchNormalization())           
    model.add(Conv2D(filters=1, kernel_size=(1, 1),
                activation='relu',
                padding='same', data_format='channels_last'))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape=(None, 12, 20, 17*number_selected_columns, 1))    
    print(model.summary())
    return model

In [ ]:
mod = create_model(number_selected_columns=12, optimizer='adam', kernel_size=(9, 9), filters=8, merge_mode="concat")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 12, 20, 204, 16)   46720     
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 20, 204, 16)   64        
_________________________________________________________________
bidirectional_4 (Bidirection (None, 12, 20, 204, 16)   124480    
_________________________________________________________________
batch_normalization_4 (Batch (None, 12, 20, 204, 16)   64        
_________________________________________________________________
bidirectional_5 (Bidirection (None, 20, 204, 16)       124480    
_________________________________________________________________
batch_normalization_5 (Batch (None, 20, 204, 16)       64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 204, 1)       

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=2)
mod.fit(X_train_reshaped, y_train_reshaped, batch_size = 8, epochs=100, verbose=2, validation_split=0.25, callbacks=[early_stopping])

In [ ]:
yhat = mod.predict(X_test_reshaped, verbose=1)

In [ ]:
# calculate error in test set

yhat_reshaped = yhat.reshape(y_test.shape[0], 340*12)
y_test_reshaped=  y_test_reshaped.reshape(y_test_reshaped.shape[0], 340*12)
testScore = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
print('Test Score: %.2f RMSE' % (testScore))